# Function Apps - On container apps!
This template enables the provisioning of a function app within an Azure Container App environment, streamlining deployment with customizable parameters. The template deploys key resources: 
- Storage account for essential data storage
- Application insights for monitoring
- Connection to the Azure Container Registry (ACR) for image management. 
- Configures a user-assigned managed identity with role assigments for secure ACR access. 
- Containerized function app on Linux

Please ensure an Container Environment is created before proceeding with the deployment, see [/scenario-container-app-environment.ipynb](../container-app-environment/scenario-container-app-environment.ipynb).

## Prerequisites
- Azure CLI installed
- Bicep CLI installed
- Python
- Jupyter Notebook or JupyterLab installed
- Container app environment ([See scenario-container-app-environment.ipynb](../container-app-environment/scenario-container-app-environment.ipynb))


## Setup Environment
1. Make an .env file, configure the settings on your needs: 

In [ ]:
%%writefile .env
SUBSCRIPTION_ID=<subscription_id>
RESOURCE_GROUP=<resource_group>
LOCATION=northeurope

TEMPLATE_FILE = "main.bicep"
PARAMETERS_FILE = "main.bicepparam"
MANAGED_ENVIRONMENT_NAME=my-container-app-env
MANAGED_ENVIRONMENT_RESOURCE_GROUP=container-apps
REGISTRY_NAME=lenndewoltentestacr
REGISTRY_RESOURCE_GROUP=container-apps

2. Install `python-dotenv` if you haven't already. You can install it using pip:

In [ ]:
!pip install python-dotenv

3. Load the settings in the environments:

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

subscription_id = os.getenv('SUBSCRIPTION_ID')
resource_group = os.getenv('RESOURCE_GROUP')
location = os.getenv('LOCATION')

deployment_name = "func-deployment"
template_file = os.getenv('TEMPLATE_FILE')
parameters_file = os.getenv('PARAMETERS_FILE')

managed_environment_name = os.getenv('MANAGED_ENVIRONMENT_NAME')
managed_environment_resource_group = os.getenv('MANAGED_ENVIRONMENT_RESOURCE_GROUP')
registry_name = os.getenv('REGISTRY_NAME')
registry_resource_group = os.getenv('REGISTRY_RESOURCE_GROUP')

print(f"Subscription ID: {subscription_id}")
print(f"Resource Group: {resource_group}")
print(f"Location: {location}")

print(f"Managed Environment Name: {managed_environment_name}")
print(f"Managed Environment Resource Group: {managed_environment_resource_group}")
print(f"Registry Name: {registry_name}")
print(f"Registry Resource Group: {registry_resource_group}")

4. Login and set default Azure subscription

In [ ]:
!az login
!az account set --subscription  {subscription_id}

5. Create an Azure Resource group:

In [ ]:
!az group create --name {resource_group} --location {location}

## Deploy template
1. Set the bicep parameters, adjust the settings when needed:

In [ ]:
%%writefile {parameters_file}

using '#{template_file}#'

param containerAppEnvName = '#{managed_environment_name}#'
param containerAppEnvRG = '#{managed_environment_resource_group}#'

param applicationName = uniqueString('#{resource_group}#')

param registry = {
  name: '#{registry_name}#'
  resourceGroup: '#{registry_resource_group}#'
}

2. Seed container registry with the docker image

In [ ]:
source = 'docker.io/lenndewolten/azurefunctions:quickstart-isolated8-v1.0.0'
target = 'azurefunctions:quickstart-isolated8-v1.0.0'

!az acr import --name {registry_name} --source {source} --image {target} --force

3. Replace tokens in `.bicepparam` file

In [ ]:
%run "../../../utils/placeholder_replacer.py"

path = f'{parameters_file}'
replace_placeholders_in_file(path)

4. Deploy the template:

In [ ]:
!az deployment group create --resource-group {resource_group} --template-file {template_file}  --parameters {parameters_file} --name {deployment_name}

5. Open the app!

In [ ]:
import os
import webbrowser

fqdn = os.popen(f"az deployment group show --resource-group {resource_group} --name {deployment_name} --query \"properties.outputs.fqdn.value\" --output tsv").read().strip()

fqdn = f"{fqdn}/api/HttpExample"
print(f"FQDN: {fqdn}")

webbrowser.open_new(fqdn)